In [1]:
############################################################
# Data taken from electoralcommission.org.uk on 15/05/15   #
#                                                          #
# Additional data about each constituancy to be downloaded #
# from democraticdashboard.com/data                        #
# --> Note that ALL of the data is hard coded into the     #
#     source code of the site!                             #
############################################################

# Import all libraries and establish version information
%pylab inline

import pandas as pd
import sqlite3
import numpy as np
import sys

print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

Populating the interactive namespace from numpy and matplotlib
Python version 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Pandas version 0.15.2


In [125]:
# NOTE:: Files are encoded in Latin1
# Read in and clean up the data [set the index on collated_results to PANo to match constituancy_results]
#     This cell does not need to be run unless you would like to alter the way that the data from the two csv files has
#     been collated.
constituancy_results = pd.DataFrame.from_csv("2015-UK-General-election-data-collated-results-Constituency.csv",
                                             encoding='latin1')

constituancy_results = constituancy_results[['Constituency Name',
                                             'Constituency ID',
                                             'Constituency Type',
                                             'Electorate',
                                             'Valid Votes']]
constituancy_results.rename(columns={'Constituency ID': 'ID',
                                     'Constituency Type': 'Type'},
                            inplace=True)
collated_results = pd.DataFrame.from_csv("2015-UK-General-election-data-collated-results-Results.csv",
                                         encoding='latin1')
collated_results.rename(columns={'Description on ballot paper': 'Party',
                                'Constituency Name': 'Constituency'},
                        inplace=True)
collated_results = collated_results[['Forename',
                                     'Surname',
                                     'Party',
                                     'Constituency',
                                     'Votes']]
joined = pd.merge(constituancy_results, collated_results, how='outer', left_index=True, right_index=True)
joined = joined[['Constituency',
                 'Surname',
                 'Forename',
                 'Party',
                 'Electorate',
                 'Valid Votes',
                 'Votes']]

def true_perc(votes, electorate):
    return float("{0:.2f}".format(votes / electorate * 100))

joined['True_percentage'] = np.vectorize(true_perc)(joined['Votes'], joined['Electorate'])

# The party names are provided in a non-standardised way so this rather ugly if block sorts that out!

parties = sorted(joined['Party'].unique().tolist())
l = dict()

for p in parties:
    if 'Conservative' in p:
        l[p] = 'Conservative'
    elif 'SDLP' in p:
        l[p] = p
    elif 'Socialist Labour Party' in p:
        l[p] = p
    elif 'Labour' in p:
        l[p] = 'Labour'
    elif p.startswith('UK'):
        l[p] = 'UKIP'
    elif 'Green Party' in p:
        l[p] = 'Green'
    elif 'Liberal Democrat' in p:
        l[p] = 'Liberal Democrat'
    elif 'Communist' in p:
        l[p] = 'Communist'
    elif 'The Peace Party' in p:
        l[p] = 'The Peace Party'
    elif p.startswith('English Democrats'):
        l[p] = 'English Democrat'
    elif 'People Before' in p:
        l[p] = 'People Before Profit'
    elif 'Pirate' in p:
        l[p] = 'Pirate Party'
    elif 'Christian Party' in p:
        l[p] = 'Christian Party'
    elif 'Left Unity' in p:
        l[p] = 'Left Unity'
    elif 'Alliance For' in p:
        l[p] = 'Alliance For Green Socialism'
    elif 'Cannabis' in p:
        l[p] = 'Cannabis is safer than alchohol'
    elif p.startswith('A '):
        l[p] = 'A Voice for the Region'
    elif p.startswith('Liberal'):
        l[p] = 'Liberal Democrat'
    elif 'Vapers' in p:
        l[p] = 'Vapers in Power'
    else:
        l[p] = p

joined['Party']= joined['Party'].map(l)

joined.to_csv('cleaned_election_data_2015.csv')

In [126]:
master = pd.read_csv('cleaned_election_data_2015.csv')
master.head()

,PANo,Constituency,Surname,Forename,Party,Electorate,Valid Votes,Votes,True_percentage
0,1,Aberavon,Kinnock,Stephen,Labour,49821,31523,15416,30.94
1,1,Aberavon,Bush,Peter,UKIP,49821,31523,4971,9.98
2,1,Aberavon,Yi He,Edward,Conservative,49821,31523,3742,7.51
3,1,Aberavon,Higgitt,Duncan,Plaid Cymru - The Party of Wales,49821,31523,3663,7.35
4,1,Aberavon,Clarke,Helen,Liberal Democrat,49821,31523,1397,2.80


In [127]:
party_totals = master.groupby('Party')['Votes'].sum().order(ascending=False)
party_stats = pd.DataFrame(party_totals)

electorate = 0
for c in list(master.Constituency.unique()):
    electorate += int(master[master.Constituency == c].Electorate.head(1))
print(electorate)

def party_perc(votes):
    return float("{0:.4f}".format(votes / electorate * 100))

party_stats['Percentage of Population'] = np.vectorize(party_perc)(party_stats['Votes'])
party_stats[:10]

46726629


,Votes,Percentage of Population
Party,,
Conservative,11299969,24.1831
Labour,9347304,20.0042
UKIP,3869864,8.2819
Liberal Democrat,2419558,5.1781
Scottish National Party (SNP),1454436,3.1126
Green,1157613,2.4774
Democratic Unionist Party - D.U.P.,184260,0.3943
Sinn Fï¿½in,176232,0.3772
Plaid Cymru - The Party of Wales,166564,0.3565


In [114]:
party_stats['Percentage of Population'].sum()

65.69680000000001

In [145]:
for p in list(master.Party.unique()):
    modal_name = list(master[master.Party == p].Forename.mode())
    if len(modal_name) > 1:
        modal_names = ', '.join(modal_name)
        print('{} are the most common names for {}.'.format(modal_names, p))
    elif modal_name:
        print('{} is the most common name for {}.'.format(modal_name[0], p))
    else:
        pass
    
master.Forename.value_counts()[:10]

David, John are the most common names for Labour.
David is the most common name for UKIP.
David is the most common name for Conservative.
John is the most common name for Plaid Cymru - The Party of Wales.
David is the most common name for Liberal Democrat.
John is the most common name for Independent.
David is the most common name for Green.
Dave is the most common name for Trade Unionist and Socialist Coalition.
Angus, John, Neil, Stewart, Stuart are the most common names for Scottish National Party (SNP).
Jim is the most common name for Scottish Trade Unionist and Socialist Coalition.
Nick is the most common name for The Official Monster Raving Loony Party.
Ian, Jim are the most common names for Democratic Unionist Party - D.U.P..
Danny  is the most common name for Ulster Unionist Party.
David is the most common name for English Democrat.
David, Michael are the most common names for no description.
Chris is the most common name for Independence from Europe.
Paul is the most common na

David      124
John       103
Paul        82
Mark        68
Peter       66
Richard     59
Andrew      58
Chris       56
James       53
Ian         48
dtype: int64

In [144]:
byParty = master.set_index('Party')
byParty[byParty.Constituency.str.contains('Altrincham')]

,PANo,Constituency,Surname,Forename,Electorate,Valid Votes,Votes,True_percentage
Party,,,,,,,,
Conservative,9,Altrincham & Sale West,Brady,Graham,72004,50517,26771,37.18
Labour,9,Altrincham & Sale West,Wright,James,72004,50517,13481,18.72
Liberal Democrat,9,Altrincham & Sale West,Brophy,Jane,72004,50517,4235,5.88
UKIP,9,Altrincham & Sale West,Frost,Christopher,72004,50517,4047,5.62
Green,9,Altrincham & Sale West,Robertson-Brown,Nick,72004,50517,1983,2.75


In [139]:
# Gutted for these guys...
byParty[byParty.True_percentage < 0.03]

,PANo,Constituency,Surname,Forename,Electorate,Valid Votes,Votes,True_percentage
Party,,,,,,,,
Republican Socialist,49,Bermondsey & Old Southwark,Freeman,Steve,80604,51424,20,0.02
no description,587,Uxbridge & Ruislip South,Jackson,James,70634,44811,14,0.02
no description,627,Witney,Handley,Nathan,79767,58482,12,0.02


In [142]:
byParty.loc['British National Party']

,PANo,Constituency,Surname,Forename,Electorate,Valid Votes,Votes,True_percentage
Party,,,,,,,,
British National Party,80,Boston & Skegness,West,Robert,67834,43339,119,0.18
British National Party,88,Braintree,Hooks,Paul,73557,50283,108,0.15
British National Party,136,Charnwood,Duffy,Cathy,77269,52261,489,0.63
British National Party,175,Dagenham & Rainham,Culnane,Tess,69049,43050,151,0.22
British National Party,317,Hornchurch & Upminster,Borg,Paul,79331,55236,193,0.24
British National Party,344,Kingswood,Lake,Julie,67992,48125,164,0.24
British National Party,438,Old Bexley & Sidcup,Finch,Nicola,66035,46748,218,0.33
British National Party,482,Rotherham,Walker,Adam,63698,37823,225,0.35
